# 1. Import Dependencies

In [1]:
!pip install opencv-python

  Using cached opencv_python-4.9.0.80-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached opencv_python-4.9.0.80-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (62.2 MB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)


In [2]:
# Import opencv
import cv2 

# Import uuid
import uuid

# Import Operating System
import os

# Import time
import time

# 2. Define Images to Collect

In [3]:
labels = ['thumbsup', 'thumbsdown', 'thankyou', 'livelong']
number_imgs = 5
# thumbsup,thumbsdown,thankyou,livelong

# 3. Setup Folders 

In [4]:
IMAGES_PATH = os.path.join('Tensorflow', 'workspace', 'images', 'collectedimages')

In [5]:
if not os.path.exists(IMAGES_PATH):
    if os.name == 'posix':
        !mkdir -p {IMAGES_PATH}
    if os.name == 'nt':
         !mkdir {IMAGES_PATH}
for label in labels:
    path = os.path.join(IMAGES_PATH, label)
    if not os.path.exists(path):
        !mkdir {path}

# 4. Capture Images

In [ ]:
import sys
import cv2
import os
import uuid
from PyQt5.QtWidgets import (QApplication, QWidget, QLabel, QPushButton, QVBoxLayout, 
                             QLineEdit, QInputDialog)
from PyQt5.QtCore import QTimer, Qt
from PyQt5.QtGui import QImage, QPixmap

# Configuration
IMAGES_PATH = 'Tensorflow/workspace/images/collectedimages'  # Replace with your actual path
os.makedirs(IMAGES_PATH, exist_ok=True)

class ImageCollectorApp(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()
        
        self.cap = cv2.VideoCapture(0)
        if not self.cap.isOpened():
            print("Error: Could not open camera.")
            self.close()

        self.timer = QTimer()
        self.timer.timeout.connect(self.update_frame)
        self.timer.start(20)  # Update the frame every 20ms

        self.countdown_timer = QTimer()
        self.countdown_timer.timeout.connect(self.update_countdown)

    def initUI(self):
        self.setWindowTitle('Image Collector')
        
        self.layout = QVBoxLayout()
        
        self.label = QLabel(self)
        self.layout.addWidget(self.label)
        
        self.class_names_entry = QLineEdit(self)
        self.class_names_entry.setPlaceholderText("Enter class names, separated by commas")
        self.layout.addWidget(self.class_names_entry)
        
        self.repeat_count_entry = QLineEdit(self)
        self.repeat_count_entry.setPlaceholderText("Enter number of images per class")
        self.layout.addWidget(self.repeat_count_entry)

        self.countdown_entry = QLineEdit(self)
        self.countdown_entry.setPlaceholderText("Enter countdown seconds")
        self.layout.addWidget(self.countdown_entry)
        
        self.capture_button = QPushButton('Start Capture', self)
        self.capture_button.clicked.connect(self.start_capture)
        self.layout.addWidget(self.capture_button)

        self.countdown_label = QLabel(self)
        self.countdown_label.setAlignment(Qt.AlignCenter)
        self.layout.addWidget(self.countdown_label)

        self.status_label = QLabel(self)
        self.status_label.setAlignment(Qt.AlignCenter)
        self.layout.addWidget(self.status_label)
        
        self.setLayout(self.layout)
        self.show()

    def update_frame(self):
        ret, frame = self.cap.read()
        if ret:
            self.current_frame = frame
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            h, w, ch = frame_rgb.shape
            bytes_per_line = ch * w
            qt_img = QImage(frame_rgb.data, w, h, bytes_per_line, QImage.Format_RGB888)
            pixmap = QPixmap.fromImage(qt_img)
            self.label.setPixmap(pixmap)

    def start_capture(self):
        class_names = self.class_names_entry.text().split(',')
        self.class_names = [name.strip() for name in class_names if name.strip()]
        try:
            self.repeat_count = int(self.repeat_count_entry.text())
            self.countdown_seconds = int(self.countdown_entry.text())
        except ValueError:
            print("Please enter valid numbers for repeat count and countdown seconds.")
            return

        if not self.class_names or self.repeat_count <= 0 or self.countdown_seconds <= 0:
            print("Please enter valid class names, repeat count, and countdown seconds.")
            return
        
        self.current_class_index = 0
        self.current_repeat_count = 0
        self.current_countdown = self.countdown_seconds
        self.start_next_capture()

    def start_next_capture(self):
        if self.current_class_index < len(self.class_names):
            self.class_name = self.class_names[self.current_class_index]
            if self.current_repeat_count < self.repeat_count:
                self.current_countdown = self.countdown_seconds
                self.countdown_label.setText(str(self.current_countdown))
                self.status_label.setText(f"Class: {self.class_name} ({self.current_repeat_count+1}/{self.repeat_count})")
                self.countdown_timer.start(1000)  # Update countdown every second
            else:
                self.current_class_index += 1
                self.current_repeat_count = 0
                self.start_next_capture()
        else:
            self.countdown_label.setText("Capture Complete")
            self.status_label.setText("")
            self.current_class_index = 0
            self.current_repeat_count = 0

    def update_countdown(self):
        if self.current_countdown > 0:
            self.current_countdown -= 1
            self.countdown_label.setText(str(self.current_countdown))
        else:
            self.countdown_timer.stop()
            self.capture_image()
            self.current_repeat_count += 1
            self.start_next_capture()

    def capture_image(self):
        if not self.class_name:
            print("Please enter a class name.")
            return
        
        imgname = os.path.join(IMAGES_PATH, self.class_name, f'{self.class_name}.{uuid.uuid1()}.jpg')
        os.makedirs(os.path.dirname(imgname), exist_ok=True)
        cv2.imwrite(imgname, self.current_frame)
        print(f"Image saved: {imgname}")
        
    def closeEvent(self, event):
        self.cap.release()
        event.accept()

if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = ImageCollectorApp()
    sys.exit(app.exec_())


# 5. Image Labelling

In [1]:
!pip install --upgrade pyqt5 lxml

In [6]:
LABELIMG_PATH = os.path.join('Tensorflow', 'labelimg')

In [9]:
if not os.path.exists(LABELIMG_PATH):
    !mkdir {LABELIMG_PATH}
    !git clone https://github.com/tzutalin/labelImg {LABELIMG_PATH}

In [11]:
if os.name == 'posix':
    !cd {LABELIMG_PATH} && make qt5py3
if os.name =='nt':
    !cd {LABELIMG_PATH} && pyrcc5 -o libs/resources.py resources.qrc

pyrcc5 -o libs/resources.py resources.qrc


In [12]:
!cd {LABELIMG_PATH} && python labelImg.py

Image:/home/toofan/Desktop/New Folder/Tensorflow-Object-Detection/Tensorflow/workspace/images/train/livelong/livelong.4e83e93e-169a-11ef-9504-dc46286ccfd2.jpg -> Annotation:/home/toofan/Desktop/New Folder/Tensorflow-Object-Detection/Tensorflow/workspace/images/train/livelong/livelong.4e83e93e-169a-11ef-9504-dc46286ccfd2.xml
Image:/home/toofan/Desktop/New Folder/Tensorflow-Object-Detection/Tensorflow/workspace/images/train/livelong/livelong.9bfd0e90-1698-11ef-9504-dc46286ccfd2.jpg -> Annotation:/home/toofan/Desktop/New Folder/Tensorflow-Object-Detection/Tensorflow/workspace/images/train/livelong/livelong.9bfd0e90-1698-11ef-9504-dc46286ccfd2.xml
Image:/home/toofan/Desktop/New Folder/Tensorflow-Object-Detection/Tensorflow/workspace/images/train/livelong/livelong.9e5ff274-1698-11ef-9504-dc46286ccfd2.jpg -> Annotation:/home/toofan/Desktop/New Folder/Tensorflow-Object-Detection/Tensorflow/workspace/images/train/livelong/livelong.9e5ff274-1698-11ef-9504-dc46286ccfd2.xml
Image:/home/toofan/Des

Image:/home/toofan/Desktop/New Folder/Tensorflow-Object-Detection/Tensorflow/workspace/images/train/thumbsup/thumbsup.36acb5d4-169a-11ef-9504-dc46286ccfd2.jpg -> Annotation:/home/toofan/Desktop/New Folder/Tensorflow-Object-Detection/Tensorflow/workspace/images/train/thumbsup/thumbsup.36acb5d4-169a-11ef-9504-dc46286ccfd2.xml
Image:/home/toofan/Desktop/New Folder/Tensorflow-Object-Detection/Tensorflow/workspace/images/train/thumbsup/thumbsup.390fb646-169a-11ef-9504-dc46286ccfd2.jpg -> Annotation:/home/toofan/Desktop/New Folder/Tensorflow-Object-Detection/Tensorflow/workspace/images/train/thumbsup/thumbsup.390fb646-169a-11ef-9504-dc46286ccfd2.xml
Cancel creation.
Image:/home/toofan/Desktop/New Folder/Tensorflow-Object-Detection/Tensorflow/workspace/images/train/thumbsup/thumbsup.783c1c76-1698-11ef-9504-dc46286ccfd2.jpg -> Annotation:/home/toofan/Desktop/New Folder/Tensorflow-Object-Detection/Tensorflow/workspace/images/train/thumbsup/thumbsup.783c1c76-1698-11ef-9504-dc46286ccfd2.xml
Image

# 6. Move them into a Training and Testing Partition

# OPTIONAL - 7. Compress them for Colab Training

In [7]:
TRAIN_PATH = os.path.join('Tensorflow', 'workspace', 'images', 'train')
TEST_PATH = os.path.join('Tensorflow', 'workspace', 'images', 'test')
ARCHIVE_PATH = os.path.join('Tensorflow', 'workspace', 'images', 'archive.tar.gz')

In [8]:
!tar -czf {ARCHIVE_PATH} {TRAIN_PATH} {TEST_PATH}